<a href="https://colab.research.google.com/github/code-lover636/Movie-Recommeder/blob/master/MovieRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import pandas as pd

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
cr_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/tmdb_5000_credits.csv')
mv_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/tmdb_5000_movies.csv')

In [71]:
df = mv_df.merge(cr_df, on="title")

# Required columns: genres, keywords, id, title, overview, popularity,  release_date, cast, crew
df = df[["genres", "keywords", "id", "title", "overview", "popularity",  "release_date", "cast", "crew"]]
df.head(3)

,genres,keywords,id,title,overview,popularity,release_date,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",206647,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [72]:
df["keywords"][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [73]:
df.isnull().sum()

genres          0
keywords        0
id              0
title           0
overview        3
popularity      0
release_date    1
cast            0
crew            0
dtype: int64

In [74]:
df.dropna(inplace=True)
# df.duplicated().sum() = 0 no duplicate values

<ipython-input-74-12eb1f54e46b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [75]:
import ast
def convert(obj):
    return [dic['name'].replace(" ","") for dic in ast.literal_eval(obj) ]


In [76]:
df['genres'] = df['genres'].apply(convert)
df['keywords'] = df['keywords'].apply(convert)
df['cast'] = df['cast'].apply(convert)
df['crew'] = df['crew'].apply(convert)
df['overview'] = df['overview'].apply(lambda x: x.split())

In [78]:
df["tags"] = df["genres"] + df["keywords"] + df["overview"] + df["cast"] + df["crew"]
df = df[['id', 'title', 'tags']]

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, unitedstates–mexicob..."
4805,72766,Newlyweds,"[Comedy, Romance, A, newlywed, couple's, honey..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TVMovie, date, loveat..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [79]:
def textProcess(obj):
    obj = " ".join(obj)
    obj = obj.lower()
    return obj

In [80]:
df["tags"] = df["tags"].apply(textProcess)

0       action adventure fantasy sciencefiction cultur...
1       adventure fantasy action ocean drugabuse exoti...
2       action adventure crime spy basedonnovel secret...
3       action crime drama thriller dccomics crimefigh...
4       action adventure sciencefiction basedonnovel m...
                              ...                        
4804    action crime thriller unitedstates–mexicobarri...
4805    comedy romance a newlywed couple's honeymoon i...
4806    comedy drama romance tvmovie date loveatfirsts...
4807    when ambitious new york attorney sam is sent t...
4808    documentary obsession camcorder crush dreamgir...
Name: tags, Length: 4805, dtype: object